In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# numpy tensors
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    
    # forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # loss
    loss = np.square(y_pred - y).sum()
    print('Loss at round {}: {}'.format(t, loss))
    
    # backprop: compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

In [ ]:
# pytorch tensors
import torch

dtype = torch.float
device = torch.device('cpu')

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    
    # forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # loss
    loss = (y_pred - y).pow(2).sum().item()
    print('Loss at round {}: {}'.format(t, loss))
    
    # backprop: compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

In [ ]:
# autograd
dtype = torch.float
device = torch.device('cpu')

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    
    # forward pass: compute predicted y
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # loss
    loss = (y_pred - y).pow(2).sum()
    print('Loss at round {}: {}'.format(t, loss.item()))
    
    # backprop: compute gradients of w1 and w2 with respect to loss
    loss.backward()
    
    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

In [ ]:
# define new autograd functions
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

dtype = torch.float
device = torch.device('cpu')

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    
    relu = MyReLU.apply
    
    # forward pass: compute predicted y
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # loss
    loss = (y_pred - y).pow(2).sum()
    print('Loss at round {}: {}'.format(t, loss.item()))
    
    # backprop: compute gradients of w1 and w2 with respect to loss
    loss.backward()
    
    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

In [ ]:
# tensorflow static graph
import tensorflow as tf

N, D_in, H, D_out = 64, 1000, 100, 10

x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

loss = tf.reduce_sum((y - y_pred) ** 2.0)

grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    
    for _ in range(500):
        loss_value, _, _ = session.run([loss, new_w1, new_w2], feed_dict={x: x_value, y: y_value})
        print(loss_value)

In [ ]:
# pytorch nn
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(torch.nn.Linear(D_in, H), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(H, D_out))

learning_rate = 1e-4

for i in range(500):
    # forward
    y_pred = model(x)
    
    # get loss
    criterion = torch.nn.MSELoss(size_average=False)
    loss = criterion(y_pred, y)
    print('{} loss in round {}'.format(loss.item(), i))
    
    # zero gradients
    model.zero_grad()
    
    # backward
    loss.backward()
    
    # update weights
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

In [ ]:
# pytorch optim
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(torch.nn.Linear(D_in, H), 
                            torch.nn.ReLU(), 
                            torch.nn.Linear(H, D_out))

learning_rate = 1e-4
optimizer = torch.optim.Adam(lr=learning_rate, params=model.parameters())

for i in range(500):
    # forward
    y_pred = model(x)
    
    # get loss
    criterion = torch.nn.MSELoss(size_average=False)
    loss = criterion(y_pred, y)
    print('{} loss in round {}'.format(loss.item(), i))
    
    # zero gradients
    optimizer.zero_grad()
    
    # backward
    loss.backward()
    
    # update weights
    optimizer.step()

In [ ]:
# pytorch custom nn modules
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        
        self.l1 = torch.nn.Linear(D_in, H)
        self.r = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.r(x)
        x = self.l2(x)
        
        return x
        
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)

learning_rate = 1e-4
optimizer = torch.optim.Adam(lr=learning_rate, params=model.parameters())

for i in range(500):
    # forward
    y_pred = model(x)
    
    # get loss
    criterion = torch.nn.MSELoss(size_average=False)
    loss = criterion(y_pred, y)
    print('{} loss in round {}'.format(loss.item(), i))
    
    # zero gradients
    optimizer.zero_grad()
    
    # backward
    loss.backward()
    
    # update weights
    optimizer.step()

In [ ]:
# pytorch dynamic graphs
import random

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        
        self.l1 = torch.nn.Linear(D_in, H)
        self.r = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(H, D_out)
        self.hidden = torch.nn.Linear(H, H)
        
    def forward(self, x):
        x = self.l1(x)
        x = self.r(x)
        
        for _ in range(random.randint(0, 3)):
            x = self.hidden(x)
            x = self.r(x)
            
        x = self.l2(x)
        
        return x
        
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)

learning_rate = 1e-4
optimizer = torch.optim.Adam(lr=learning_rate, params=model.parameters())

for i in range(500):
    # forward
    y_pred = model(x)
    
    # get loss
    criterion = torch.nn.MSELoss(size_average=False)
    loss = criterion(y_pred, y)
    print('{} loss in round {}'.format(loss.item(), i))
    
    # zero gradients
    optimizer.zero_grad()
    
    # backward
    loss.backward()
    
    # update weights
    optimizer.step()